In [1]:
import Graph_Amazon_Movies as gam
import Amazon_Movie_Parser as amp
import numpy as np
import pandas as pd
from networkx.algorithms import bipartite
import networkx as nx
import Amazon_Movie_Parser as prs
import datetime
from numpy import savetxt
import Computations_debug as comp
import random

## Random Walk

## VALIDATION TESTS

In [2]:
'''
file_name = 'data/movies.txt'; n_movies = 40000
n_movies = 40000
n_movies_v = 5000; start_index_v = n_movies
ref_user_idx = 735
walk_steps = 20; beta=0.15; top_neighbor=40
run_part = 'ALL'  # run_part = 'Test' # this doesn't run the walk again
'''
def one_shot_run(file_name, n_movies, n_movies_v, start_index_v, ref_user_idx, walk_steps, beta, top_neighbor):
    # >>>>>>>>>>>  Preprocessing  <<<<<<<<<<<<<<<
    # >>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<
    start_time = datetime.datetime.now()
    grpComp = comp.GraphComp()

    grp = gam.Graph_Amazon()
    #file_name='data/movies.txt'; n_movies=40000; 
    prs_out='dictionary'

    max_connected_gr_amazon_movies = grpComp.Create_Bipartite_Giant_Component(grp, file_name, n_movies, prs_out)

    bottom_nodes, top_nodes = bipartite.sets(max_connected_gr_amazon_movies)

    grpComp.health_check(grp, max_connected_gr_amazon_movies, bottom_nodes, top_nodes)
    
    ####   P calculations
    P_norm, R, R_zero = grpComp.Generate_P_Matrix(max_connected_gr_amazon_movies, bottom_nodes, top_nodes)

    node_list, bottom_node_list, top_node_list, n_reviews, n_users, n_movies = \
    grpComp.generate_node_lists(max_connected_gr_amazon_movies, bottom_nodes, top_nodes)

    movie_indexes, user_indexes = grpComp.movie_user_indexes(node_list, top_node_list)
    
    # R_matrice
    #R = random_walk(P_norm, R, R_zero, beta=0.15, n_steps=2)
    # for the matrix, n_steps=3, n_movies=40K ... calculation_time=12:56 minutes
    # the advantage of using the matrix is, we don't have to wait for the calculations of others if we would like to
    # check the results when we change the top_similarity and ref_user_idx
    # but since we do the walk for all the users each time, it takes too much time...
    
    # >>>>>>>>>>>>>  Random Walk  <<<<<<<<<<<<<<<
    # >>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<
    # R_vector
    #ref_user_idx=735
    R_vector = grpComp.random_walk_vector(P_norm, R[ref_user_idx], R_zero[ref_user_idx], beta=beta, n_steps=walk_steps)
    # for the vector, n_steps=3, n_movies=40K ... calculation time is 0.66 seconds...
    
    # >>>>>>>>>>>  Summary  <<<<<<<<<<<<<<<
    # >>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<
    # R_matrice
    #df_summary, movie_list_ref, movie_by_reference_user = similarity_check(R, ref_user_idx=3, top_similarity=40)
    
    # R_vector
    # R_vector
    df_summary, movie_list_ref, movie_by_reference_user, user_similarity_top_non_zero = \
    grpComp.similarity_check_vector(R_vector, max_connected_gr_amazon_movies, movie_indexes,\
                                    ref_user_idx, node_list, top_similarity=top_neighbor)

    df_summary_ratio = \
    grpComp.similarity_summary_ratio(df_summary, movie_list_ref, max_connected_gr_amazon_movies, \
                                     movie_by_reference_user)

    ##pd.set_option('display.height', 500)
    #pd.set_option('display.max_rows', 300)
    #print(df_summary)

    # >>>>>>>>>>>  Validation Tests <<<<<<<<<<<<<<<
    # >>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<
    grpComp_v = comp.GraphComp()
    grp_v = gam.Graph_Amazon()
    # start_index_v >> here index means record, in reality index=9*start_index
    #file_name_v='data/movies.txt'; start_index_v=40000; n_movies_v=5000; prs_out_v='dictionary'
    file_name_v=file_name; prs_out_v='dictionary'

    max_connected_gr_amazon_movies_VAL = \
    grpComp_v.Create_Bipartite_Giant_Component_VAL(grp_v, file_name_v, start_index_v, n_movies_v, prs_out_v)

    bottom_nodes_v, top_nodes_v = bipartite.sets(max_connected_gr_amazon_movies_VAL)

    grpComp_v.health_check(grp_v, max_connected_gr_amazon_movies_VAL, bottom_nodes_v, top_nodes_v)

    # We will need P_VAL to compare the results between our prediction and the reality
    # Well.... actually using edge check might be more convenient...
    #P_norm_v, R_v, R_zero_v = Generate_P_Matrix(max_connected_gr_amazon_movies_VAL, bottom_nodes_v, top_nodes_v)

    node_list_v, bottom_node_list_v, top_node_list_v, n_reviews_v, n_users_v, n_movies_v = \
    grpComp_v.generate_node_lists(max_connected_gr_amazon_movies_VAL, bottom_nodes_v, top_nodes_v)

    movie_indexes_v, user_indexes_v = grpComp_v.movie_user_indexes(node_list_v, top_node_list_v)
    
    # what we are trying is, we will see if a movie in the new set was reviewed with our top neighbors or not
    # if the movie satisfies the condition we will check, then we will guess as 
    # there is an edge between the user and the movie. 
    # we will create the R vector for this user and see the accuracy of our prediction
    # we will predict edge for each movie in the second batch for the ref-user

    #movie_FOR_reference_user = []
    movie_FOR_reference_user = top_node_list_v

    '''
    # we will only use the NEW movies in the NEW batch to make a prediction
    for movie_v in top_node_list_v:
        if movie_v not in top_node_list:
            movie_FOR_reference_user.append(movie_v)
    '''
    
    # here we will use the training parameters to make our prediction
    user_name_val = node_list[ref_user_idx]
    print('User name checked: {}'.format(user_name_val))
    #ref_user_idx_val=9149
    #R_vector = grpComp_v.random_walk_vector(P_norm, R[ref_user_idx_val], R_zero[ref_user_idx_val], beta=0.15, n_steps=20)

    #movie_FOR_reference_user = top_node_list_v

    df_summary_v, movie_list_ref_v, movie_by_reference_user_v, user_similarity_top_non_zero = \
    grpComp_v.similarity_check_vector_VAL(R_vector, movie_indexes, ref_user_idx, node_list, node_list_v, \
    movie_FOR_reference_user, max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL, top_similarity=top_neighbor)
    
    # we will check the values in max_connected_gr_amazon_movies not VAL because our reference is the first one
    df_summary_ratio_v = \
    grpComp_v.similarity_summary_ratio_VAL(df_summary_v, movie_list_ref_v, \
                             max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL)
    
    end_time = datetime.datetime.now()
    
    print('n_reviews for modeling = {} .. n_movies for testing = {}'.format(n_movies, n_movies_v))
    print('top_neighbors picked = {} .. steps walked = {} .. beta = {}'.format(top_neighbor, walk_steps, beta))
    print('Calculation time-ALL: {}'.format(end_time-start_time))
    
    return max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL, P_norm

In [3]:
file_name = 'data/movies.txt'; n_movies = 40000
n_movies_v = 5000; start_index_v = n_movies
ref_user_idx = 652
walk_steps = 20; beta=0.15; top_neighbor=40
#run_part = 'ALL'  # run_part = 'Test' # this doesn't run the walk again

In [4]:
max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL, P_norm=\
one_shot_run(file_name, n_movies, n_movies_v, start_index_v, ref_user_idx, walk_steps, beta, top_neighbor)

Calculation time-Crete graph from the movies file: 0:00:35.898668
Is connected: True ... Is bipartite: True
BEFORE: # of nodes: 30082 ... # of edges: 38233
P to_numpy_matrix: 2020-05-13 17:31:57.796992
P normalize: 2020-05-13 17:32:02.785807
P finalize: 2020-05-13 17:32:12.842134
Calculation time-P matrix generation from the graph: 0:00:15.039797
Calculation time-user-movie indexes: 0:00:12.331289
Calculation time-steps walked: 0:00:09.571186


# of users in the evaluation: 40
Create_Bipartite_VALIDATION is running...
Calculation time-Crete graph from the movies file: 0:00:00.646005
Is connected: True ... Is bipartite: True
BEFORE: # of nodes: 3412 ... # of edges: 4634
Calculation time-user-movie indexes: 0:00:00.110914
User name checked: A35ZK3M8L9JUPX


n_reviews for modeling = 788 .. n_movies for testing = 47
top_neighbors picked = 40 .. steps walked = 20 .. beta = 0.15
Calculation time-ALL: 0:01:24.184175


In [219]:
# save the necessary info to files to avoid repeating the calculations - PART-1
# max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL, P_norm
'''
MovieGraph_1 = nx.to_numpy_matrix(max_connected_gr_amazon_movies)
MovieGraph_1_val = nx.to_numpy_matrix(max_connected_gr_amazon_movies)

from tempfile import TemporaryFile
outfile_MovieGraph_1 = TemporaryFile()
'''

# if we don't convert to string, if check doesn't recognize numpy but np as we defined earlier
# however, some people might choose to give something else than np as a short name
# converting to string fixes this conflict
def save_objects_to_file(input_obj, file_name_to_save = 'outfile'):
    # we might remove the import from here
    #from tempfile import TemporaryFile
    #outfile = TemporaryFile()
    #file_name_to_save = 'outfile'
    if str(type(input_obj)) == "<class 'networkx.classes.graph.Graph'>":
        #print(type(input_obj))
        grp_sample = nx.to_numpy_matrix(input_obj)
        np.save(file_name_to_save, grp_sample)
        print('your graph is saved in matrix format')
    elif str(type(input_obj)) == "<class 'numpy.matrix'>":
        #print(type(input_obj))
        np.save(file_name_to_save, input_obj)
        print('your matrix is saved')
    elif str(type(input_obj)) == "<class 'pandas.core.frame.DataFrame'>":
        df_FINAL.to_csv(file_name_to_save)
        print('your dataframe is saved')
    else:
        str_type_graph = "<class 'networkx.classes.graph.Graph'>"
        str_type_matrix = "<class 'numpy.matrix'>"
        str_type_dataframe = "<class 'pandas.core.frame.DataFrame'>"
        print('Be sure that you are using NUMPY as np and PANDAS as pd')
        print('The type of the object is not supported: {}'.format(str(type(input_obj))))
        print('Supported types:\n{}\n{}\n{}'.format(str_type_graph, str_type_matrix, str_type_dataframe))

def load_objects_from_file(output_obj = 'matrix', file_name_to_read = 'inputfile'):
    # we might remove the import from here
    #from tempfile import TemporaryFile
    #outfile = TemporaryFile()
    #file_name_to_save = 'outfile'
    if output_obj == "graph":
        #print(type(input_obj))
        file_name_to_read = 'Test_1/outfile_max_connected_gr_amazon_movies.npy'
        matrix_sample = np.load(file_name_to_read)
        grp_loaded = nx.from_numpy_matrix(matrix_sample)
        return grp_loaded
        print('your graph is loaded')
    elif output_obj == "matrix":
        #print(type(input_obj))
        file_name_to_read = 'Test_1/P_norm.npy'
        matrix_loaded = np.load(file_name_to_read)
        return matrix_loaded
        print('your matrix is loaded')
    elif output_obj == "dataframe":
        file_name_to_read = 'Test_1/outfile_df_FINAL'
        df_FINAL_loaded = pd.read_csv(file_name_to_read) 
        return df_FINAL_loaded
        print('your dataframe is loaded')
    else:
        str_type_graph = "graph"
        str_type_matrix = "matrix"
        str_type_dataframe = "dataframe"
        print('Be sure that you are using: NUMPY as np ... PANDAS as pd ... NETWORKX as nx')
        print('The type of the object is not supported: {}'.format(str(type(output_obj))))
        print('Supported types:\n{}\n{}\n{}'.format(str_type_graph, str_type_matrix, str_type_dataframe))
        
def save_parameter_used(parameter_list, file_name='parameter_list.csv'):
    # put it to the top not in the function maybe
    import csv
    #with open(file_name, 'w', newline='') as myfile:
    with open(file_name, 'w', newline='\n') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(parameter_list)
        

In [220]:
parameter_list = ["file_name = 'data/movies.txt'", "n_movies = 40000", "n_movies_v = 5000", "ref_user_idx = 652",
                 "walk_steps = 40", "beta=0.15", "top_neighbor=40", "n_test_users = 100", "threshold=10"]

save_parameter_used(parameter_list, file_name='parameter_list.csv')

In [184]:
str(type(df_FINAL))

"<class 'pandas.core.frame.DataFrame'>"

In [192]:
save_objects_to_file(df_FINAL)

your dataframe is saved


In [213]:
test1_load = load_objects_from_file('graphaascav')

Be sure that you are using: NUMPY as np ... PANDAS as pd ... NETWORKX as nx
The type of the object is not supported: <class 'str'>
Supported types:
graph
matrix
dataframe


In [210]:
grp_load1 = load_objects_from_file('graph')

In [212]:
len(grp_load1.edges)

38233

In [207]:
mx1 = load_objects_from_file('matrix')

In [209]:
mx1.shape

(30082, 30082)

In [204]:
df1 = load_objects_from_file('dataframe')

In [206]:
df1

,Unnamed: 0,user,movie,prediction,reality,ngbr_ratio,tot_rev_m,n_old_edge,n_new_edge
0,0,A175GIISFGGO31,B003Y5GMNC,0,0,0.0,25,2,1
1,1,A175GIISFGGO31,B00004CZR1,0,0,0.0,200,2,1
2,2,A175GIISFGGO31,B00005Y6YM,0,0,0.0,24,2,1
3,3,A175GIISFGGO31,B00106S9TM,0,0,0.0,25,2,1
4,4,A175GIISFGGO31,B00005Y6YP,0,0,0.0,18,2,1
...,...,...,...,...,...,...,...,...,...
4695,4695,A3AGIOKDN01U4U,B00005Y6YS,0,0,2.5,23,1,1
4696,4696,A3AGIOKDN01U4U,B000EQ5V86,0,0,7.5,189,1,1
4697,4697,A3AGIOKDN01U4U,B00007ELMT,0,0,0.0,34,1,1
4698,4698,A3AGIOKDN01U4U,B00005Y6YV,0,0,0.0,4,1,1


In [160]:
if str(type(P_norm)) == "<class 'numpy.matrix'>":
    print('yes')

yes


In [86]:
start_time = datetime.datetime.now()

grp = gam.Graph_Amazon(); grpComp = comp.GraphComp(); grpComp_v = comp.GraphComp()
#grp = gam.Graph_Amazon(); grpComp = comp.GraphComp(debug_mode='On'); grpComp_v = comp.GraphComp(debug_mode='On')

print('Preparation starts at {}'.format(datetime.datetime.now()))
bottom_nodes, top_nodes = bipartite.sets(max_connected_gr_amazon_movies)
bottom_nodes_v, top_nodes_v = bipartite.sets(max_connected_gr_amazon_movies_VAL)

n_test_users = 100; walk_steps = 40; beta = 0.15; top_neighbor = 40; threshold=10
n_nodes = len(max_connected_gr_amazon_movies.nodes); n_nodes_v = len(max_connected_gr_amazon_movies_VAL.nodes)
n_movies = len(list(top_nodes)); n_users = len(list(bottom_nodes))
n_movies_v = len(list(top_nodes_v)); n_users_v = len(list(bottom_nodes_v))

node_list = list(max_connected_gr_amazon_movies.nodes); node_list_v = list(max_connected_gr_amazon_movies_VAL.nodes)
top_node_list = list(top_nodes); top_node_list_v = list(top_nodes_v); 

movie_indexes, user_indexes = grpComp.movie_user_indexes(node_list, top_node_list)
movie_indexes_v, user_indexes_v = grpComp_v.movie_user_indexes(node_list_v, top_node_list_v)

# pick some users to test randomly
#test_users = random.sample(bottom_nodes, k = n_test_users)

# what if we pick randomly from common users? (instead of selecting the test users from the old dataset)
lst_common_users = list(set(bottom_nodes) - (set(bottom_nodes) - set(bottom_nodes_v)))
test_users = random.sample(lst_common_users, k = n_test_users)

# a table test_users * new_movies (movies in the new dataset-can also exist in the first dataset)
# but obviously the edges are totally new in the new dataset
lst_users = []
lst_movies = []
lst_predict = []
lst_real = []
lst_ratios = []
lst_total_reviews_for_movies = []
lst_total_OLD_edges_for_users = []
lst_total_NEW_edges_for_users = []

# test a single user
#test_users = ['A15BIF2J5V7IHZ']

movie_FOR_reference_user = list(top_nodes_v)
for i, user in enumerate(test_users):
    ref_user_idx = node_list.index(user) 
    R = np.zeros(n_nodes); R[ref_user_idx] = 1
    R_zero = R.copy()
    print('Random walk starts for {}-{} at {}'.format(ref_user_idx, user, datetime.datetime.now()))
    #R_vector = grpComp.random_walk_vector(P_norm, R[ref_user_idx], R_zero[ref_user_idx], beta=beta, n_steps=walk_steps)
    R_vector = grpComp.random_walk_vector(P_norm, R, R_zero, beta=beta, n_steps=walk_steps)
    
    '''
    # similarity_check_vector_VAL is doing the entire check for both the modeling dataset and the new dataset
    # a user can have an edge in the new dataset, too. since we predict 1 user-movie connection at a time,
    # we use the new data, as well. this approach is open to the discussion.
    # if we consider a missing edge, this approach should be fine but if we totally need to find ALL edges at once,
    # then we shouldn't consider the new edges in calculations
    # so we don't do re-modeling from scratch but we take advantage of the existing (new) edges to evaluate
    df_summary, movie_list_ref, movie_by_reference_user, similar_users = \
    grpComp.similarity_check_vector(R_vector, max_connected_gr_amazon_movies, movie_indexes,\
                                    ref_user_idx, node_list, top_similarity = top_neighbor)
    '''
    
    print('similarity_check_vector_VAL starts for {}-{} at {}'.format(ref_user_idx, user, datetime.datetime.now()))
    ### check below if needed.. or if above needed..
    df_summary_v, movie_list_ref_v, movie_by_reference_user_v, user_similarity_top_non_zero = \
    grpComp_v.similarity_check_vector_VAL(R_vector, movie_indexes, ref_user_idx, node_list, node_list_v, \
    movie_FOR_reference_user, max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL, top_similarity=top_neighbor)
    
    '''
    # this is to see the ratio in the modeling dataset, not required for the prediction phase
    # but can be checked to see the performance in the modeling dataset
    df_summary_ratio = \
    grpComp.similarity_summary_ratio(df_summary, movie_list_ref, max_connected_gr_amazon_movies, \
                                     movie_by_reference_user)
    '''

    print('similarity_summary_ratio_VAL starts for {}-{} at {}'.format(ref_user_idx, user, datetime.datetime.now()))

    print('**********************************************')
    df_summary_ratio_v = \
    grpComp_v.similarity_summary_ratio_VAL(df_summary_v, movie_list_ref_v, \
                             max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL)
    
    # we will iterate in the new movies
    for mv_new in top_node_list_v:
        # ratio in the summary view for a specific movie (will always return 1 record since we give the mean 
        # in the summmary view, so values[0])
        ratio_m = df_summary_ratio_v.query("movie=='" + mv_new + "'").values[0].tolist()[1]

        # how many times this movie reiewed 
        review_m = df_summary_ratio_v.query("movie=='" + mv_new + "'").values[0].tolist()[2]
        
        lst_users.append(user)
        lst_movies.append(mv_new)
        predict_calc = 1 if ratio_m > threshold else 0
        lst_predict.append(predict_calc)
        real_m = max_connected_gr_amazon_movies_VAL.has_edge(user, mv_new)
        lst_real.append(1 if real_m==True else 0)
        lst_ratios.append(ratio_m)
        lst_total_reviews_for_movies.append(review_m)
        edge_old = len(max_connected_gr_amazon_movies.edges(user))
        edge_new = len(max_connected_gr_amazon_movies_VAL.edges(user))
        lst_total_OLD_edges_for_users.append(edge_old)
        lst_total_NEW_edges_for_users.append(edge_new)
    
grpComp_v.movie_user_compare_datasets(top_nodes_v, top_nodes, bottom_nodes_v, bottom_nodes)

dict_FINAL = {'user': lst_users, 'movie': lst_movies, 'prediction': lst_predict, 'reality': lst_real, 
                'ngbr_ratio': lst_ratios, 'tot_rev_m': lst_total_reviews_for_movies, 
              'n_old_edge': lst_total_OLD_edges_for_users, 'n_new_edge': lst_total_NEW_edges_for_users}

df_FINAL = pd.DataFrame(dict_FINAL)
    
end_time = datetime.datetime.now()
print('**********************************************')
print('Calculation time for ALL-Predictions: {}'.format(end_time-start_time))


Preparation starts at 2020-05-13 19:48:28.498190
Calculation time-user-movie indexes: 0:00:12.686243
Calculation time-user-movie indexes: 0:00:00.112553
Random walk starts for 26900-A175GIISFGGO31 at 2020-05-13 19:48:41.952395
Calculation time-steps walked: 0:00:13.118035
similarity_check_vector_VAL starts for 26900-A175GIISFGGO31 at 2020-05-13 19:48:55.070759


similarity_summary_ratio_VAL starts for 26900-A175GIISFGGO31 at 2020-05-13 19:48:58.471694
**********************************************
Random walk starts for 17668-A22CI3VI6XA586 at 2020-05-13 19:48:58.757014
Calculation time-steps walked: 0:00:13.215350
similarity_check_vector_VAL starts for 17668-A22CI3VI6XA586 at 2020-05-13 19:49:11.972824


similarity_summary_ratio_VAL starts for 17668-A22CI3VI6XA586 at 2020-05-13 19:49:15.531799
**********************************************
Random walk starts for 356-A14NA0W8ESGDSI at 2020-05-13 19:49:15.823140
Calculation time-steps walked: 0:00:13.216907
similarity_check_vector_VAL s

Calculation time-steps walked: 0:00:14.422526
similarity_check_vector_VAL starts for 1008-A3DWUM6SN3N3NR at 2020-05-13 19:55:45.993566


similarity_summary_ratio_VAL starts for 1008-A3DWUM6SN3N3NR at 2020-05-13 19:55:49.557755
**********************************************
Random walk starts for 2244-A22BPC7LRSGSYQ at 2020-05-13 19:55:49.855128
Calculation time-steps walked: 0:00:13.784482
similarity_check_vector_VAL starts for 2244-A22BPC7LRSGSYQ at 2020-05-13 19:56:03.639927


similarity_summary_ratio_VAL starts for 2244-A22BPC7LRSGSYQ at 2020-05-13 19:56:07.195411
**********************************************
Random walk starts for 1460-A82R4WKX62LEK at 2020-05-13 19:56:07.458275
Calculation time-steps walked: 0:00:14.477724
similarity_check_vector_VAL starts for 1460-A82R4WKX62LEK at 2020-05-13 19:56:21.936387


similarity_summary_ratio_VAL starts for 1460-A82R4WKX62LEK at 2020-05-13 19:56:25.270249
**********************************************
Random walk starts for 30-A2582KMXL

Random walk starts for 26024-A2Z9XBCYT3AHHU at 2020-05-13 20:02:43.387758
Calculation time-steps walked: 0:00:14.503391
similarity_check_vector_VAL starts for 26024-A2Z9XBCYT3AHHU at 2020-05-13 20:02:57.891677


similarity_summary_ratio_VAL starts for 26024-A2Z9XBCYT3AHHU at 2020-05-13 20:03:01.233987
**********************************************
Random walk starts for 28503-AHZFJ1Z2A3FKQ at 2020-05-13 20:03:01.497211
Calculation time-steps walked: 0:00:14.451356
similarity_check_vector_VAL starts for 28503-AHZFJ1Z2A3FKQ at 2020-05-13 20:03:15.948870


similarity_summary_ratio_VAL starts for 28503-AHZFJ1Z2A3FKQ at 2020-05-13 20:03:19.283921
**********************************************
Random walk starts for 17467-A7WCPQWOJR4XI at 2020-05-13 20:03:19.542562
Calculation time-steps walked: 0:00:14.377881
similarity_check_vector_VAL starts for 17467-A7WCPQWOJR4XI at 2020-05-13 20:03:33.920984


similarity_summary_ratio_VAL starts for 17467-A7WCPQWOJR4XI at 2020-05-13 20:03:37.251299
***



similarity_summary_ratio_VAL starts for 21142-A636KL3G56E02 at 2020-05-13 20:09:55.640673
**********************************************
Random walk starts for 17742-A2B5BJM25KAQMB at 2020-05-13 20:09:55.915582
Calculation time-steps walked: 0:00:14.426192
similarity_check_vector_VAL starts for 17742-A2B5BJM25KAQMB at 2020-05-13 20:10:10.342208


similarity_summary_ratio_VAL starts for 17742-A2B5BJM25KAQMB at 2020-05-13 20:10:13.689834
**********************************************
Random walk starts for 17744-AKCE3Y6JBTAK8 at 2020-05-13 20:10:13.950053
Calculation time-steps walked: 0:00:14.488901
similarity_check_vector_VAL starts for 17744-AKCE3Y6JBTAK8 at 2020-05-13 20:10:28.439472


similarity_summary_ratio_VAL starts for 17744-AKCE3Y6JBTAK8 at 2020-05-13 20:10:31.761577
**********************************************
Random walk starts for 7132-A2WSI8HOWHFDOT at 2020-05-13 20:10:32.019201
Calculation time-steps walked: 0:00:14.508494
similarity_check_vector_VAL starts for 7132-A

Calculation time-steps walked: 0:00:14.381481
similarity_check_vector_VAL starts for 1340-A32JKNQ6BABMQ2 at 2020-05-13 20:17:04.835177


similarity_summary_ratio_VAL starts for 1340-A32JKNQ6BABMQ2 at 2020-05-13 20:17:08.202304
**********************************************
Random walk starts for 24425-A3TUQ1JB7ZZNZG at 2020-05-13 20:17:08.461232
Calculation time-steps walked: 0:00:14.428600
similarity_check_vector_VAL starts for 24425-A3TUQ1JB7ZZNZG at 2020-05-13 20:17:22.890313


similarity_summary_ratio_VAL starts for 24425-A3TUQ1JB7ZZNZG at 2020-05-13 20:17:26.249751
**********************************************
Random walk starts for 17647-A16IUW5089L51J at 2020-05-13 20:17:26.520488
Calculation time-steps walked: 0:00:14.511952
similarity_check_vector_VAL starts for 17647-A16IUW5089L51J at 2020-05-13 20:17:41.032858


similarity_summary_ratio_VAL starts for 17647-A16IUW5089L51J at 2020-05-13 20:17:44.353978
**********************************************
Random walk starts for 499

In [ ]:
# save the necessary info to files to avoid repeating the calculations - PART-2
# df_FINAL

In [87]:
df_FINAL.query("prediction==1 or reality==1")
#df_FINAL.query("n_new_edge>0")

,user,movie,prediction,reality,ngbr_ratio,tot_rev_m,n_old_edge,n_new_edge
10,A175GIISFGGO31,B00005Y6YJ,0,1,0.0,91,2,1
84,A22CI3VI6XA586,B00005LL26,1,1,92.5,345,1,1
110,A14NA0W8ESGDSI,B003UAMWWQ,0,1,0.0,163,1,1
156,A2HIWDD2TDXS19,5553600715,0,1,2.5,379,1,1
225,A3CUDLCGBSG881,B00005LL26,1,1,92.5,345,1,1
265,ASJ54MITON1NO,0767805534,0,1,10.0,119,10,1
274,ASJ54MITON1NO,B00004CZRQ,1,0,15.0,375,10,1
319,APGCYBTLF5VMJ,B00005LL26,1,1,92.5,345,1,1
330,A81P47EIXM8HA,B00004CZR1,1,0,12.5,200,7,2
335,A81P47EIXM8HA,B00004CZR2,1,0,12.5,200,7,2


In [146]:
# threshold can be adjusted since the ratio is already provided
#threshold = 10
def final_summary(df_FINAL):
    tp = len(df_FINAL.query("prediction==1 and reality==1").values)
    fp = len(df_FINAL.query("prediction==1 and reality==0").values)
    tn = len(df_FINAL.query("prediction==0 and reality==0").values)
    fn = len(df_FINAL.query("prediction==0 and reality==1").values)
    
    acc = (tp + tn)/(tp + tn + fp + fn)
    tpr = tp/(tp + fn) # sensitivity, recall
    tnr = tn/(tn + fp) # sprecifity 
    fpr = fp/(fp + tn)
    ppv = tp/(tp + fp)  # precision - positive predictive value
    f1 = (2*tp)/(2*tp + fp + fn)
    print('tp={} .. fp={} .. tn={} .. fn={}'.format(tp, fp, tn, fn, acc))
    print('acc={} .. tpr={} .. f1={}'.format(acc, tpr, f1))
    print('fpr={} .. tnr={} .. ppv={}'.format(fpr, tnr, ppv))
    
final_summary(df_FINAL)

tp=52 .. fp=56 .. tn=4505 .. fn=87
acc=0.9695744680851064 .. tpr=0.37410071942446044 .. f1=0.42105263157894735
fpr=0.012278009208506906 .. tnr=0.9877219907914931 .. ppv=0.48148148148148145


In [122]:
lst_prediction_val = df_FINAL.loc[:, 'prediction'].values.tolist()
lst_true_val = df_FINAL.loc[:, 'reality'].values.tolist()
lst_ratio_val = df_FINAL.loc[:, 'ngbr_ratio'].values.tolist()

from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(lst_true_val, lst_ratio_val)  #, pos_label=2

print('fpr={}'.format(fpr))
print('tpr={}'.format(tpr))
print('thresholds={}'.format(thresholds))

fpr=[0.00000000e+00 2.19250164e-04 2.19250164e-04 6.57750493e-04
 6.57750493e-04 8.77000658e-04 1.31550099e-03 1.53475115e-03
 3.06950230e-03 5.48125411e-03 1.22780092e-02 2.91602719e-02
 6.73098005e-02 1.56325367e-01 3.80837536e-01 1.00000000e+00]
tpr=[0.         0.         0.21582734 0.21582734 0.22302158 0.22302158
 0.22302158 0.22302158 0.27338129 0.32374101 0.37410072 0.42446043
 0.48920863 0.60431655 0.76978417 1.        ]
thresholds=[101.  100.   92.5  45.   37.5  35.   22.5  20.   17.5  15.   12.5  10.
   7.5   5.    2.5   0. ]


In [127]:
fpr, tpr, thresholds = metrics.roc_curve(lst_true_val, lst_ratio_val)

In [130]:
import matplotlib.pylot as plt

ModuleNotFoundError: No module named 'matplotlib.pylot'

In [ ]:
'''
# try this for matplotlib
pip uninstall matplotlib
python3 -m pip install matplotlib
'''

In [139]:
#!/usr/bin/env python
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
#import matplotlib.pylot as plt

fpr, tpr, thresholds = roc_curve(lst_true_val, lst_ratio_val)
roc_auc = roc_auc_score(lst_true_val, lst_ratio_val)
print(roc_auc)


'''
# there is a problem with my matplotlib...
plt.figure()
plt.plot(fpr, tpr, label="Random Walk - ROC" % roc_auc)
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1.0])
plt.ylim([0,0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('False Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.savefig('ROC_fig')
plt.show()
'''

0.7787450372961882


'\n# there is a problem with my matplotlib...\nplt.figure()\nplt.plot(fpr, tpr, label="Random Walk - ROC" % roc_auc)\nplt.plot([0, 1], [0, 1], \'r--\')\nplt.xlim([0, 1.0])\nplt.ylim([0,0, 1.05])\nplt.xlabel(\'False Positive Rate\')\nplt.ylabel(\'False Positive Rate\')\nplt.title(\'ROC\')\nplt.legend(loc="lower right")\nplt.savefig(\'ROC_fig\')\nplt.show()\n'

In [140]:
import matplotlib.pylot as plt


ModuleNotFoundError: No module named 'matplotlib.pylot'

In [150]:
# we can reorganize the results depending on the threshold
# we copy the original dataframe not to lose the original results but since we just change the prediction,
# it is ok to change in the original dataframe, too
df_FINAL_copy = df_FINAL.copy()
threshold = 2.5
for i in range(len(df_FINAL_copy)):
    if df_FINAL_copy.loc[i, 'ngbr_ratio'] > threshold:
        df_FINAL_copy.loc[i, 'prediction'] = 1
    else:
        df_FINAL_copy.loc[i, 'prediction'] = 0
        

final_summary(df_FINAL_copy)

tp=84 .. fp=713 .. tn=3848 .. fn=55
acc=0.8365957446808511 .. tpr=0.60431654676259 .. f1=0.1794871794871795
fpr=0.15632536724402543 .. tnr=0.8436746327559745 .. ppv=0.1053952321204517


In [84]:
lst_common_users = list(set(bottom_nodes) - (set(bottom_nodes) - set(bottom_nodes_v)))
print(lst_common_users)

['A2WKKTVYS5H61M', 'A7P9LPMSBAAME', 'A5Q1FK0FWU7DH', 'A3B08DCVOC1BDI', 'A3FC0DAS1XUUY8', 'APRAUZ9Z7X4PI', 'A1PJZKEZQGREDY', 'A18OUQEK7IZ2F2', 'A7Y6AVS576M03', 'A3AW8XP45YVIIZ', 'ATEHUNVE9M3E0', 'A1NDOQNA06X8N8', 'A30AQ325LCZGKH', 'A3FZ06XRKW5JC5', 'A2OTX4X3895YXT', 'A11PTCZ2FM2547', 'A3MWXVT0ENVQHT', 'ACDIH3SOZ3Z56', 'A5TZXWU8AALIC', 'A35YWA2V4QHOJ3', 'AYG1U47VFZ165', 'A2GPN2VKS8E77C', 'A1UBSDCCZNLELH', 'A2MMIZ653FBNPT', 'A2WSI8HOWHFDOT', 'A27YYPZ140WG56', 'ASHT96DUVMF1S', 'A2Y8DTZHS7U028', 'A1TOH8S8I1VT9R', 'A2PCXXBTYRQ2FF', 'A1EOLRV216BXJ9', 'A2ZZHMT58ZMVCZ', 'A28GCEHD00JFZG', 'AJ8CBMFSPRKQU', 'A1BR8RV8ZH540P', 'AB0W9GTNGHSI3', 'A1RECBDKHVOJMW', 'A3JTBJC5WSEZ7Q', 'A28NZJQZUCQOJK', 'AZOW89D0NXMUT', 'A2ABDL2SLHIO8N', 'A18RQ2URRGIIEA', 'A23KDYA7YGPEWW', 'A1K0XHNAJCUCMO', 'A3UCNXHX2PB3KQ', 'APVKV29EDMS5H', 'A1B38S1YJR1PQD', 'A1TW9ZGRDQQZ2Y', 'A3M5O6UHXO9IBU', 'A461VTLW9G9YB', 'AB1UMC7SB9D7Z', 'AA5NDJEMF2CF7', 'ADXJBL8H8VWVT', 'A3P98J5DZ00A75', 'A1POFVVXUZR3IQ', 'A21XVS00AIENLX', 'A2IKUDK

In [79]:
#pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
df_FINAL

,user,movie,prediction,reality,ngbr_ratio,tot_rev_m,n_old_edge,n_new_edge
0,A15BIF2J5V7IHZ,B003Y5GMNC,0,0,0.0,25,1,2
1,A15BIF2J5V7IHZ,B00004CZR1,0,0,2.5,200,1,2
2,A15BIF2J5V7IHZ,B00005Y6YM,0,0,0.0,24,1,2
3,A15BIF2J5V7IHZ,B00106S9TM,0,0,0.0,25,1,2
4,A15BIF2J5V7IHZ,B00005Y6YP,0,0,0.0,18,1,2
5,A15BIF2J5V7IHZ,B007XF0X4I,0,0,0.0,45,1,2
6,A15BIF2J5V7IHZ,B00004CZR2,0,0,2.5,200,1,2
7,A15BIF2J5V7IHZ,1557832854,0,0,0.0,21,1,2
8,A15BIF2J5V7IHZ,0788600222,0,0,0.0,12,1,2
9,A15BIF2J5V7IHZ,B00005Y6YQ,0,0,0.0,22,1,2


In [67]:
df_summary_ratio_v.query("movie=='B000ZLFALI'")

,movie,ratio_similar,review_count
26,B000ZLFALI,5.0,587


In [46]:
# ratio in the summary view for a specific movie (will always return 1 record since we give the mean 
# in the summmary view, so values[0])
df_summary_ratio_v.query("movie=='" + "B003Y5GMNC'").values[0].tolist()[1]

# how many times this movie reiewed 
df_summary_ratio_v.query("movie=='" + "B003Y5GMNC'").values[0].tolist()[2]

25

In [39]:
# df_summary shows the relations between the top-neighbors and the movies in the new dataset
#df_summary_v.query("movie=='B00007ELMT'")
df_summary_v.query("is_reviewed==True")
#df_summary_v.query("is_reviewed==True").values.shape
#df_summary_v.query("is_reviewed==True").values[0].tolist()
#df_summary_v

,user,movie,user_id,movie_id,is_reviewed,score,n_review
61,ABO2ZI2Y5DQ9T,B000ZLFALS,2888,2464,True,3.0,587
65,ABO2ZI2Y5DQ9T,B000EQ5V8G,2888,2305,True,1.0,189
73,ABO2ZI2Y5DQ9T,B000ZLFALI,2888,1504,True,3.0,587
90,ABO2ZI2Y5DQ9T,B000EQ5V86,2888,889,True,1.0,189
108,ATXL536YX71TR,B000ZLFALS,5,2464,True,2.0,587
120,ATXL536YX71TR,B000ZLFALI,5,1504,True,2.0,587
207,A16CZRQL23NOIW,B00775YIO0,489,3001,True,4.0,12
211,A16CZRQL23NOIW,0784011532,489,2102,True,4.0,56
224,A16CZRQL23NOIW,6303160565,489,2837,True,4.0,20
225,A16CZRQL23NOIW,B00005LL26,489,505,True,5.0,345


In [8]:
m='B00004CZR1'
len(list(max_connected_gr_amazon_movies.edges([m])))

0

In [9]:
#user_name_val = node_list[ref_user_idx]
#print('User: {} .. \nedges: {}'.format(user_name_val, max_connected_gr_amazon_movies_VAL.edges(user_name_val)))

In [10]:
list(max_connected_gr_amazon_movies_VAL.edges(user_name_val))[0][1]

NameError: name 'user_name_val' is not defined

In [32]:
user_name_val='ABO2ZI2Y5DQ9T'
lst_new_movies_reviewed = list(max_connected_gr_amazon_movies_VAL.edges(user_name_val))
for i in range(len(lst_new_movies_reviewed)):
    new_mov = lst_new_movies_reviewed[i][1]
    #new_mov_summary = df_summary_ratio_v.query("movie=='" + new_mov + "'").values.tolist()
    new_mov_summary = df_summary_ratio_v.query("movie=='" + new_mov + "'").values[0]
    print(new_mov_summary)
    
'''    
# we don't fetch the old data into a dataframe, we do the calculations through the graph
for i in range(len(lst_new_movies_reviewed)):
    new_mov = lst_new_movies_reviewed[i][1]
    new_mov_summary = df_summary_ratio.query("user_movie=='" + new_mov + "'").values.tolist()
    print(new_mov_summary)
 
#true_values=len(df_summary.query("movie=='B003AI2VGA' and is_reviewed==True"))
'''

['B000EQ5V86' 2.5 189]
['B000ZLFALI' 7.5 587]
['B000EQ5V8G' 2.5 189]
['B000ZLFALS' 7.5 587]


'    \n# we don\'t fetch the old data into a dataframe, we do the calculations through the graph\nfor i in range(len(lst_new_movies_reviewed)):\n    new_mov = lst_new_movies_reviewed[i][1]\n    new_mov_summary = df_summary_ratio.query("user_movie==\'" + new_mov + "\'").values.tolist()\n    print(new_mov_summary)\n \n#true_values=len(df_summary.query("movie==\'B003AI2VGA\' and is_reviewed==True"))\n'

In [26]:
new_mov_summary[0]

['B000ZLFALS', 7.5, 587]

In [ ]:
grpComp_v.movie_user_compare_datasets(top_nodes_v, top_nodes, bottom_nodes_v, bottom_nodes)

In [ ]:
max_connected_gr_amazon_movies.get_edge_data('A1I7QGUDP043DG', 'B000ZLFALI')

In [ ]:
max_connected_gr_amazon_movies_VAL.get_edge_data('A1I7QGUDP043DG', 'B00005LL26')

In [53]:
max_connected_gr_amazon_movies_VAL.has_edge('A1I7QGUDP043DG', 'B00005LL26')

False

In [ ]:
grpComp_v.movie_user_compare_datasets(top_nodes_v, top_nodes, bottom_nodes_v, bottom_nodes)

In [56]:
len(max_connected_gr_amazon_movies_VAL.edges('A130VTEXX25T2G'))

0

In [12]:
max_connected_gr_amazon_movies.edges('A130VTEXX25T2G')

EdgeDataView([('A130VTEXX25T2G', 'B005GLHO48')])

In [ ]:
len(max_connected_gr_amazon_movies.edges('A1I7QGUDP04'))

In [ ]:
#max_connected_gr_amazon_movies_VAL.edges()